# Unlock Cost Savings with New Scale-to-Zero Feature in SageMaker Inference


This demo notebook demonstrate how you can scale in your SageMaker endpoint to zero instances during idle periods, eliminating the previous requirement of maintaining at least one running instance.

The new Scaling to Zero feature expands the possibilities for managing SageMaker Inference endpoints. It allows customers to configure the endpoints so they can scale to zero instances during periods of inactivity, providing an additional tool for resource management. Using this feature customers can closely match their compute resource usage to their actual needs, potentially reducing costs during times of low demand. This enhancement builds upon SageMaker's existing auto-scaling capabilities, offering more granular control over resource allocation. Customers can now configure their scaling policies to include scaling to zero, allowing for more precise management of their AI inference infrastructure. 

The Scaling to Zero feature presents new opportunities for how businesses can approach their cloud-based machine learning operations. It provides additional options for managing resources across various scenarios, from development and testing environments to production deployments with variable traffic patterns. As with any new feature, customers are encouraged to carefully evaluate how it fits into their overall architecture and operational needs, considering factors such as response times and the specific requirements of their applications.

#### Determining When to Scale Down to Zero

SageMaker's scale-to-zero capability is ideal for three scenarios:

1. **Predictable traffic patterns:** If your inference traffic is predictable and follows a consistent schedule, you can use this scaling functionality to automatically scale in to zero during periods of low or no usage. This eliminates the need to manually delete and recreate inference components/endpoints.

2. **Sporadic workloads:** For applications that experience sporadic or variable inference traffic patterns, scaling in to zero instances can provide significant cost savings. However, it's important to note that scaling out from zero instances to serving traffic is not instantaneous. During the scale-out process, any requests sent to the endpoint will fail, and these "NoCapacityInvocationFailures" will be captured in CloudWatch.

3. **Development and testing:** The scale-to-zero functionality is also beneficial when testing and evaluating new machine learning models. During model development and experimentation, you may create temporary inference endpoints to test different configurations. However, it's easy to forget to delete these endpoints when you're done. Scaling to zero ensures these test endpoints automatically scale in to zero instances when not in use, preventing unwanted charges. This allows you to freely experiment without closely monitoring infrastructure usage or remembering to manually delete endpoints. The automatic scaling to zero provides a cost-effective way to test out ideas and iterate on your machine learning solutions.
   
**Note:** Scale-to-zero is only supported when using inference components. for more information on Inference Components see “[Reduce model deployment costs by 50% on average using the latest features of Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/reduce-model-deployment-costs-by-50-on-average-using-sagemakers-latest-features/)” blog.


## Set up

In [ ]:
!pip install --force-reinstall --no-cache-dir sagemaker==2.235.2

In [ ]:
import boto3
import botocore
import sagemaker
import sys
import time
import json

sagemaker_client = boto3.client("sagemaker")
role = sagemaker.get_execution_role()
print(f"Role: {role}")

boto_session = boto3.Session()
sagemaker_session = sagemaker.session.Session(boto_session) # sagemaker session for interacting with different AWS APIs
region = sagemaker_session._region_name

model_bucket = sagemaker_session.default_bucket()  # bucket to house model artifacts

prefix = sagemaker.utils.unique_name_from_base("DEMO")
print(f"prefix: {prefix}")

## Setup your SageMaker Real-time Endpoint 

### Create a SageMaker endpoint configuration

We begin by creating the endpoint configuration and set MinInstanceCount to 0. This allows the endpoint to scale in all the way down to zero instances when not in use.

In [ ]:
# Set an unique name for our endpoint config
endpoint_config_name = f"{prefix}-llama3-8b-scale-to-zero-aas-config"
print(f"Endpoint config name: {endpoint_config_name}")

In [ ]:
# Configure variant name and instance type for hosting
variant_name = "AllTraffic"
instance_type = "ml.g5.12xlarge"
model_data_download_timeout_in_seconds = 3600
container_startup_health_check_timeout_in_seconds = 3600

min_instance_count = 0 # Minimum instance must be set to 0
max_instance_count = 3

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": min_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

### Create the SageMaker endpoint
Next, we create our endpoint using the above endpoint config

In [ ]:
# Set a unique endpoint name
endpoint_name = f"{prefix}-llama3-8b-scale-to-zero-aas-endpoint"
print(f"Endpoint name: {endpoint_name}")

sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

#### We wait for our endpoint to go InService. This step can take ~3 mins

In [ ]:
import time
# Let's see how much it takes
start_time = time.time()

while True:
    desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = desc["EndpointStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

## Create Model Builder
We use Amazon SageMaker Fast Model Loader. The feature works by streaming model weights directly from Amazon S3 to GPU accelerators, bypassing the typical sequential loading steps that contribute to deployment latency. In internal testing, this approach has shown to load large models up to 15 times faster compared to traditional methods. For more information on this feature, please refer to our example [notebook on GitHub](https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/Llama3.1/Llama3.1-70B-SageMaker-Fast-Model-Loader.ipynb)

We'll make use of the ModelBuilder class to prepare and package the model inference components. In this example, we're using the Meta-Llama-3-8B-Instruct SageMaker JumpStart.

Key configurations:
- Model: Meta-Llama-3-8B-Instruct
- Schema Builder: Defines input/output format

In [ ]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
import logging

prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"
response = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the east coast."

model_id = "meta-textgeneration-llama-3-8b-instruct"

model_builder = ModelBuilder(
    model=model_id,
    role_arn=role,
    sagemaker_session=sagemaker_session,
    schema_builder=SchemaBuilder(sample_input=prompt, sample_output=response),
    #env_vars={
    #   "OPTION_TENSOR_PARALLEL_DEGREE": "4",
    #},
    log_level=logging.WARN
)

output_path = f"s3://{model_bucket}/llama3-8b/sharding"

Note that, if you have already run the model optimization job before and the model shards are available on s3. You can just reuse the shards and skip the optimization step
```
model_builder = ModelBuilder(
             model="meta-textgeneration-llama-3-8b-instruct",
             model_metadata={
                "CUSTOM_MODEL_PATH": output_path,
            },
           schema_builder=SchemaBuilder(sample_input="Test", sample_output="Test"),
            role_arn=role,
             instance_type="ml.g5.12xlarge",
)
```

## Optimize Model for Fast Loading
Now we'll optimize the model using Fast Model Loader. This process:
1. Prepares model shards for deployment
2. Enables direct streaming from S3 to GPU
3. Pre-configures tensor parallelism settings

Note: The optimization process may take a while to complete. The optimized model will be stored in the specified S3 output path.

In [ ]:
model_builder.optimize(
    instance_type="ml.g5.12xlarge", 
    accept_eula=True, 
    output_path=output_path, 
    sharding_config={
            "OverrideEnvironment": {
                "OPTION_TENSOR_PARALLEL_DEGREE": "4"
            }
    }
)

## Build and Deploy Model
After optimization, we'll build the final model artifacts and deploy them to a SageMaker endpoint. 

Key configurations:
- Instance Type: ml.g5.12xlarge
- Memory Request: 4096 MB
- Number of Accelerators: 4 (for tensor parallelism)

In [ ]:
final_model = model_builder.build()

In [ ]:
# Make sure our model is sharded
final_model._is_sharded_model

In [ ]:
# EnableNetworkIsolation cannot be set to True since SageMaker Fast Model Loading of model requires network access.
if final_model._enable_network_isolation:
    final_model._enable_network_isolation = False
    
final_model._enable_network_isolation

#### Select the container image to use
Use the latest LMI image to take advantage of caching feature

In [ ]:
final_model.image_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.31.0-lmi13.0.0-cu124"
print(f"Image going to be used is ---- > {final_model.image_uri}")

In [ ]:
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

resources_required = ResourceRequirements(
    requests={
        "memory" : 4096,
        "num_accelerators": 4,
        "copies": 1, # specify the number of initial copies (default is 1)
    },
)

#### Deploy your model to the endpoint

Deploy your model with the model’s existing deploy method. We specify the name of our existing Real-time endpoint SageMaker will host the model on our existing endpoint, so it can starts making predictions on incoming requests.

In [ ]:
final_model.deploy(
    instance_type="ml.g5.12xlarge", 
    accept_eula=True, 
    endpoint_name=endpoint_name,
    # endpoint_logging=False, 
    resources=resources_required,
)

### Test the endpoint with a sample prompt
Now we can invoke our endpoint with sample text to test its functionality and see the model's output.

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
inference_component_name = final_model.inference_component_name
model_id = "meta-textgeneration-llama-3-8b-instruct"

In [ ]:
# create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    component_name=inference_component_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)


# Prompt to generate
messages = [
    {"role": "system", "content": "You are a helpful assistant. Be concise"},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

## Automatically Scale To Zero

## Scaling policies 

Once the endpoint is deployed and InService, you can then add the necessary scaling policies:

* A [target tracking](https://docs.aws.amazon.com/autoscaling/application/userguide/application-auto-scaling-target-tracking.html) policy that can scale in the copy count for our inference component model copies to zero, and from 1 to n. 
* A [step scaling policy](https://docs.aws.amazon.com/autoscaling/application/userguide/application-auto-scaling-step-scaling-policies.html) policy that will allow the endpoint to scale out from zero.

These policies work together to provide cost-effective scaling - the endpoint can scale to zero when idle and automatically scale out as needed to handle incoming requests.

### Scaling policy for inference components copies (target tracking)

We start with creating our target tracking policies for scaling the CopyCount of our inference component

#### Register a new autoscaling target
After you create your SageMaker endpoint and inference components, you register a new auto scaling target for Application Auto Scaling. In the following code block, you set **MinCapacity**  to **0**, which is required for your endpoint to scale down to zero

In [ ]:
aas_client = sagemaker_session.boto_session.client("application-autoscaling")
cloudwatch_client = sagemaker_session.boto_session.client("cloudwatch")

In [ ]:
# Autoscaling parameters
resource_id = f"inference-component/{inference_component_name}"
service_namespace = "sagemaker"
scalable_dimension = "sagemaker:inference-component:DesiredCopyCount"

min_copy_count = 0
max_copy_count = 8

aas_client.register_scalable_target(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    MinCapacity=min_copy_count,
    MaxCapacity=max_copy_count,
)

#### Configure Target Tracking Scaling Policy
Once you have registered your new scalable target, the next step is to define your target tracking policy. In the code example that follows, we set the TargetValue to 5. This setting instructs the auto-scaling system to increase capacity when the number of concurrent requests per model reaches or exceeds 5. Here we are taking advantage of the more granular auto scaling metric `PredefinedMetricType`: `SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution` to more accurately monitor and react to changes in inference traffic. Take a look this [blog](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-inference-launches-faster-auto-scaling-for-generative-ai-models/) for more information. 

In [ ]:
aas_client.describe_scalable_targets(
    ServiceNamespace=service_namespace,
    ResourceIds=[resource_id],
    ScalableDimension=scalable_dimension,
)

# The policy name for the target traking policy
target_tracking_policy_name = f"Target-tracking-policy-llama3-8b-scale-to-zero-aas-{inference_component_name}"

aas_client.put_scaling_policy(
    PolicyName=target_tracking_policy_name,
    PolicyType="TargetTrackingScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    TargetTrackingScalingPolicyConfiguration={
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution",
        },
        # Low TPS + load TPS
        "TargetValue": 5,  # you need to adjust this value based on your use case
        "ScaleInCooldown": 300,  # default
        "ScaleOutCooldown": 300,  # default
    },
)

Application Auto Scaling creates two CloudWatch alarms per scaling target. The first triggers scale-out actions after 30 seconds (using 3 sub-minute data point), while the second triggers scale-in after 15 minutes (using 90 sub-minute data points). The time to trigger the scaling action is usually 1–2 minutes longer than those minutes because it takes time for the endpoint to publish metrics to CloudWatch, and it also takes time for AutoScaling to react. 

### Scale out from zero policy (step scaling policy )
To enable your endpoint to scale out from zero instances, do the following:

#### Configure Step Scaling Policy
Create a step scaling policy that defines when and how to scale out from zero. This policy will add 1 model copy when triggered, enabling SageMaker to provision the instances required to handle incoming requests after being idle.  The following shows you how to define a step scaling policy. Here we have configured to scale out from 0 to 1 model copy ("ScalingAdjustment": 1), depending on your use case you can adjust ScalingAdjustment as required. 

In [ ]:
# The policy name for the step scaling policy
step_scaling_policy_name = f"Step-scaling-policy-llama3-8b-scale-to-zero-aas-{inference_component_name}"

aas_client.put_scaling_policy(
    PolicyName=step_scaling_policy_name,
    PolicyType="StepScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "MetricAggregationType": "Maximum",
        "Cooldown": 60,
        "StepAdjustments":
          [
             {
               "MetricIntervalLowerBound": 0,
               "ScalingAdjustment": 1
             }
          ]
    },
)

In [ ]:
resp = aas_client.describe_scaling_policies(
    PolicyNames=[step_scaling_policy_name],
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
)
step_scaling_policy_arn = resp['ScalingPolicies'][0]['PolicyARN']
print(f"step_scaling_policy_arn: {step_scaling_policy_arn}")

#### Create the CloudWatch alarm that will trigger our policy

Finally, create a CloudWatch alarm with the metric **NoCapacityInvocationFailures**. When triggered, the alarm initiates the previously defined scaling policy. For more information about the NoCapacityInvocationFailures metric, see [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-inference-component).

We have also set the following:
- EvaluationPeriods to 1 
- DatapointsToAlarm to 1 
- ComparisonOperator to  GreaterThanOrEqualToThreshold

This results in 1 min waiting for the step scaling policy to trigger


In [ ]:
# The alarm name for the step scaling alarm
step_scaling_alarm_name = f"step-scaling-alarm-llama3-8b-scale-to-zero-aas-{inference_component_name}"

cloudwatch_client.put_metric_alarm(
    AlarmName=step_scaling_alarm_name,
    AlarmActions=[step_scaling_policy_arn],  # Replace with your actual ARN
    MetricName='NoCapacityInvocationFailures',
    Namespace='AWS/SageMaker',
    Statistic='Maximum',
    Dimensions=[
        {
            'Name': 'InferenceComponentName',
            'Value': inference_component_name  # Replace with actual InferenceComponentName
        }
    ],
    Period=30, # Set a lower period 
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=1,
    ComparisonOperator='GreaterThanOrEqualToThreshold',
    TreatMissingData='missing'
)

## Testing the behaviour
Notice the `MinInstanceCount: 0` setting in the Endpoint configuration, which allows the endpoint to scale down to zero instances. With the scaling policy, CloudWatch alarm, and minimum instances set to zero, your SageMaker Inference Endpoint will now be able to automatically scale down to zero instances when not in use, helping you optimize your costs and resource utilization.

### IC copy count scales in to zero
We'll pause for a few minutes without making any invocations to our model. Based on our target tracking policy, when our SageMaker endpoint doesn't receive requests for about 10 to 15 minutes, it will automatically scale down to zero the number of model copies. 

In [ ]:
time.sleep(900)
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name)
print(desc)

### Endpoint's instances scale in to zero

After 10 additional minutes of inactivity, SageMaker automatically terminates all underlying instances of the endpoint, eliminating all associated costs.

In [ ]:
# after 10mins instances will scale down to 0
time.sleep(600)
# verify whether CurrentInstanceCount is zero
sagemaker_session.wait_for_endpoint(endpoint_name)

### Invoke the endpoint with a sample prompt

If we try to invoke our endpoint while instances are scaled down to zero, we get a validation error: `An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component has no capacity to process this request. ApplicationAutoScaling may be in-progress (if configured) or try to increase the capacity by invoking UpdateInferenceComponentRuntimeConfig API.`

In [ ]:
print(time.strftime("%H:%M:%S"))
# create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    component_name=inference_component_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)


# Prompt to generate
messages = [
    {"role": "system", "content": "You are a helpful assistant. Be concise"},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

### Scale out from zero kicks in
However, after 1 minutes our step scaling policy should kick in. SageMaker will then start provisioning a new instance and deploy our inference component model copy to handle requests. This demonstrates the endpoint's ability to automatically scale out from zero when needed.

In [ ]:
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name)
print(desc)

#### verify that our endpoint has succesfully scaled out from zero

In [ ]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    component_name=inference_component_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)


# Prompt to generate
messages = [
    {"role": "system", "content": "You are a helpful assistant. Be concise"},
    {"role": "user", "content": "What is deep learning?"},
]

# Generation arguments
parameters = {
    "model": model_id,  # model id is required
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = predictor.predict({"messages": messages, **parameters})

# Unpack and print response
print(chat["choices"][0]["message"]["content"].strip())

## Optionally clean up the environment

- Deregister scalable target
- Delete cloudwatch alarms
- Delete scaling policies

In [ ]:
try:
    # Deregister the scalable target for AAS
    aas_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension=scalable_dimension,
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)

# Delete CloudWatch alarms created for Step scaling policy
try:
    cloudwatch_client.delete_alarms(AlarmNames=[step_scaling_alarm_name])
    print(f"Deleted CloudWatch step scaling scale-out alarm [b]{step_scaling_alarm_name} ✅")
except cloudwatch_client.exceptions.ResourceNotFoundException:
    print(f"CloudWatch scale-out alarm [b]{step_scaling_alarm_name}[/b] not found.")


# Delete step scaling policies
print("---" * 10)

try:
    aas_client.delete_scaling_policy(
        PolicyName=step_scaling_policy_name,
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Deleted scaling policy [i green]{step_scaling_policy_name} ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scaling policy [i]{step_scaling_policy_name}[/i] not found.")

- Delete inference component
- Delete endpoint
- delete endpoint-config
- Delete model

In [ ]:
sagemaker_client.delete_inference_component(InferenceComponentName=inference_component_name)
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)